In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
#ee.Authenticate()
#geemap.update_package()

In [3]:
##converting javascript code to python (also in map interface):
#js_snippet = """
#"""
#geemap.js_snippet_to_py(js_snippet, add_new_cell = True, import_ee = False, import_geemap = False, show_map = False)

In [4]:
#list of plots in Virginia LTER dataset (not including broad creek)
ee.Initialize()

plot_pts = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-75.372740, 37.903114]), {'site_id': 'Assateague'}), #Assateague
  ee.Feature(ee.Geometry.Point([-75.671577, 37.625273]), {'site_id': 'Bellvue'}), #Bellvue
  ee.Feature(ee.Geometry.Point([-75.876094, 37.395746]), {'site_id': 'Box_Tree'}), #Box_Tree
  ee.Feature(ee.Geometry.Point([-75.652248, 37.709249]), {'site_id': 'Channel_Point'}), #Channel_Point
  ee.Feature(ee.Geometry.Point([-75.942331, 37.175055]), {'site_id': 'Cushmans'}), #Cushmans
  ee.Feature(ee.Geometry.Point([-75.940586, 37.167409]), {'site_id': 'Gator'}), #Gator
  ee.Feature(ee.Geometry.Point([-75.814954, 37.485033]), {'site_id': 'Greens_Creek'}), #Greens_Creek
  ee.Feature(ee.Geometry.Point([-75.672871, 37.451132]), {'site_id': 'Hog'}), #Hog
  ee.Feature(ee.Geometry.Point([-75.900996, 37.346190]), {'site_id': 'Indiantown'}), #Indiantown
  ee.Feature(ee.Geometry.Point([-75.558264, 37.784154]), {'site_id': 'Kegotank'}), #Kegotank
  ee.Feature(ee.Geometry.Point([-75.861733, 37.416783]), {'site_id': 'Mill_Creek'}), #Mill_Creek
  ee.Feature(ee.Geometry.Point([-75.822928, 37.468043]), {'site_id': 'N_Brownsville'}), #N_Brownsville
  ee.Feature(ee.Geometry.Point([-75.929133, 37.287549]), {'site_id': 'Oyster'}), #Oyster
  ee.Feature(ee.Geometry.Point([-75.940690, 37.181277]), {'site_id': 'Steelmans'}), #Steelmans
  ee.Feature(ee.Geometry.Point([-75.447000, 37.880000]), {'site_id': 'Wallops'}), #Wallops
  ee.Feature(ee.Geometry.Point([-75.819360, 37.481998]), {'site_id': 'Woodland_Farm'}) #Woodland_Farm
])


ga_pts = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-81.422318, 31.539096]), {'site_id': '1'}),
  ee.Feature(ee.Geometry.Point([-81.295564, 31.535967]), {'site_id': '2'}),
  ee.Feature(ee.Geometry.Point([-81.229141, 31.518847]), {'site_id': '3'}),
  ee.Feature(ee.Geometry.Point([-81.365805, 31.451791]), {'site_id': '4'}),
  ee.Feature(ee.Geometry.Point([-81.340722, 31.435814]), {'site_id': '5'}),
  ee.Feature(ee.Geometry.Point([-81.280011, 31.387962]), {'site_id': '6'}),
  ee.Feature(ee.Geometry.Point([-81.478498, 31.334813]), {'site_id': '7'}),
  ee.Feature(ee.Geometry.Point([-81.415535, 31.30824]), {'site_id': '8'}),
  ee.Feature(ee.Geometry.Point([-81.333707, 31.351518]), {'site_id': '9'}),
  ee.Feature(ee.Geometry.Point([-81.270863, 31.476948]), {'site_id': '10'})
])
##for GA, lats and longs may differ slightly between years (compare average lat/long to recorded lat/longs in each yr)

In [5]:
##landsat 5 surface reflectance images - VA LTER

roi = ee.Geometry.Point(-75.876094, 37.395746)

filteredCollectionLS5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(1999, 2017,'year')) \
    .filter(ee.Filter.calendarRange(8, 9,'month')) \
    .filterBounds(plot_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(filteredCollectionLS5.size().getInfo())
image0 = filteredCollectionLS5.first()
#print(image0.getInfo())

37


In [6]:
##landsat 8 surface reflectance images - VA LTER
filteredCollectionLS8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(1999, 2017,'year')) \
    .filter(ee.Filter.calendarRange(8, 9,'month')) \
    .filterBounds(plot_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(filteredCollectionLS8.size().getInfo())

14


In [7]:
#landsat 7 surface reflectance images - VA LTER - might not be able to use due to striping
filteredCollectionLS7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(1999, 2017,'year')) \
    .filter(ee.Filter.calendarRange(8, 9,'month')) \
    .filterBounds(plot_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(filteredCollectionLS7.size().getInfo())

61


In [8]:
##get list of image ids in filtered collection for L5, L7, and L8
#filteredCollectionLS5.aggregate_array('system:id').getInfo()
#filteredCollectionLS8.aggregate_array('system:id').getInfo()
#filteredCollectionLS7.aggregate_array('system:id').getInfo()

In [9]:
##add image number for each of the images in the filtered collection - closer to 8/15 is best

##landsat 5
image00 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20000901') 
image01 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20010803') 
image02 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20020822')
image03 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20030825') 
image04 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20040811')
image05 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20050814')
image06 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20060817') 
image07 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20070905')
image08 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20080822')
image09 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20090809')
image10 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20100828') 
image11 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20110815')

##landsat 7
image12 = ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_014034_20120926') #l7 striping is fixed in study area!

##landsat 8
image13 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20130905') 
image14 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20140807') 
image15 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20150826') 
image16 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20160812') 
image17 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20170831') 

##Make image collection for each year, then cloud mask? Rather than cloud masking individual images?

##determining cloud cover
cloudiness = image06.get('CLOUD_COVER')
#print('CLOUD_COVER: ', cloudiness.getInfo())

l8_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B5', 'B4', 'B3'], 
             'gamma': 1.5} ##be very careful with analysis of l8 vs l5 data - different band numbers!

#Map.addLayer(image15, l8_param, "l82015")
#Map.addLayer(image12, vis_param, "l72012")

props = geemap.image_props(image17)
props.getInfo()

{'CLOUD_COVER': 5.57,
 'CLOUD_COVER_LAND': 5.37,
 'EARTH_SUN_DISTANCE': 1.009305,
 'ESPA_VERSION': '2_23_0_1a',
 'GEOMETRIC_RMSE_MODEL': 8.398,
 'GEOMETRIC_RMSE_MODEL_X': 7.087,
 'GEOMETRIC_RMSE_MODEL_Y': 4.506,
 'IMAGE_DATE': '2017-08-31',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_014034_20170831_20170915_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1505475319000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2017-08-31T15:40:46.3074390Z',
 'SOLAR_AZIMUTH_ANGLE': 142.104111,
 'SOLAR_ZENITH_ANGLE': 34.529263,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 14,
 'WRS_ROW': 34,
 'system:asset_size': '492.059884 MB',
 'system:band_names': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B10',
  'B11',
  'sr_aerosol',
  'pixel_qa',
  'radsat_qa'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_014034_20170831',
 'system:index': 'LC08_014034_20170831',
 'system:time_end': '2017-08-31 15:40:46',

In [12]:
##Mapping
##Add Earth Engine dataset
Map = geemap.Map(center=[40,-100], zoom=4)

Map.addLayer(image0, vis_param, "Landsat5SR")

Map.centerObject(plot_pts, 10)
#Map.set_plot_options(add_marker_cluster = True) ##not sure what this does


##add feature collection points for each plot
Map.addLayer(plot_pts)
Map.addLayer(ga_pts)

Map

Map(center=[37.485005614872954, -75.76448856851104], controls=(WidgetControl(options=['position'], widget=HBox…

In [13]:
#extracting raster values for points; 
#from: https://developers.google.com/earth-engine/tutorials/community/extract-raster-values-for-points
#buffering points

In [85]:
## This example demonstrates the use of the pixel QA band to mask
## clouds in surface reflectance (SR) data.  It is suitable
## for use with any of the Landsat SR datasets.

##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.
def maskL5sr(image):
  ##Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  ##Get the pixel QA band.
  qa = image.select('pixel_qa')

  ##Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  ##Return the masked image, scaled to reflectance, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])

##Map the function over image collection
collection = filteredCollectionLS5 \
    .map(maskL5sr)

composite = collection.median() #what exactly is this doing? median pixel value?

##Display the results:

#Map.addLayer(composite, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, "cloudfree composite")

In [86]:
masked00 = ee.Image(maskL5sr(image00))
Map.addLayer(composite) #using cloud mask on cloudfree composite with default vis param as gray background
Map.addLayer(masked00, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, "2000 masked") #isn't perfect but good
Map.addLayer(image00, vis_param, "2000 unmasked")
#Map.addLayer(collection.first(), {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.3}, "masked???")
Map.addLayer(plot_pts)

In [87]:
##cloud- and cloud shadow-masked images - VA LTER :D
masked00 = ee.Image(maskL5sr(image00))
masked01 = ee.Image(maskL5sr(image01))
masked02 = ee.Image(maskL5sr(image02))
masked03 = ee.Image(maskL5sr(image03))
masked04 = ee.Image(maskL5sr(image04))
masked05 = ee.Image(maskL5sr(image05))
masked06 = ee.Image(maskL5sr(image06))
masked07 = ee.Image(maskL5sr(image07))
masked08 = ee.Image(maskL5sr(image08))
masked09 = ee.Image(maskL5sr(image09))
masked10 = ee.Image(maskL5sr(image10))
masked11 = ee.Image(maskL5sr(image11))
masked12 = ee.Image(maskL5sr(image12))
masked13 = ee.Image(maskL5sr(image13))
masked14 = ee.Image(maskL5sr(image14))
masked15 = ee.Image(maskL5sr(image15))
masked16 = ee.Image(maskL5sr(image16))
masked17 = ee.Image(maskL5sr(image17))

In [92]:
##Exporting pixel values - any way to do zonal averages without manually selecting - VA LTER
in_fc = plot_pts

##export data
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'masked07.csv')
#out_shp = os.path.join(out_dir, 'points.shp') ##shapefile exporting
#geemap.extract_values_to_points(in_fc, masked07, out_csv) ##projection error when exporting composites

In [57]:
##summer median composite pixel values - error when exporting values!!!
LS5comp2000 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2000, 2000,'year')) \
    .filterBounds(plot_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(LS5comp2000.size().getInfo())

collection2000 = LS5comp2000.map(maskL5sr)
composite2000 = collection2000.median()
#Map.addLayer(composite2000, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.3}, "cloudfree composite 2000")

16


In [51]:
#LS5comp2000.aggregate_array('system:id').getInfo()

In [29]:
print("hi")

hi
